# Modeling Evaporation and Transpiration
Overall goal: to write a computer program to compute the soil moisture using a daily time step, and to predict ET in the moisture balance using 
1.  Penman's (or Penman-Monteith) method and 
2. the pan evaporation method (use pan coefficient of 0.7). 


TODO:
* cleaned up until here, 
    * double check the df variable renames got fully renamed (used changed all occurances which can be risky)
    * fix capitalization on the other variables 
    * take out long pathnames and assume local files 
    * get the dataframe column names to be something meaningful
    * maybe break the data into 2 models for some of the null values to get pulled out
    *include data cleaning logic, possibly take it out later

In [3]:
#imports
import pandas as pd
import numpy as np
import os
import math
import numpy
from numpy import random
from scipy.stats import skewnorm
import plotly.graph_objects as go

In [6]:
os.path.dirname(os.getcwd())

'/mnt/c/dev'

# Data Import

### Reading in the meterologic data that will be used to train the model

In [5]:
#reading in meterologic data
waseca = pd.read_excel('Data/waseca.xls')
print("Shape of dataframne:", waseca.shape)
waseca.head()

Shape of dataframne: (62, 9)


,date,tmax,tmin,rain,pan,srad,rhmax,rhmin,wd spd
0,1985-06-01,20.3,8.5,0.51,3.81,2.64,94.0,64.0,4.41
1,1985-06-02,15.5,4.5,0.00,3.30,4.85,89.0,45.0,3.65
2,1985-05-03,18.9,5.1,0.00,4.32,5.80,89.0,39.0,2.25
3,1985-06-04,18.3,11.4,0.00,4.32,2.34,83.0,61.0,2.24
4,1985-06-05,23.4,8.1,0.00,8.64,8.51,89.0,22.0,2.53


Initial investigation/cleaning

In [ ]:
#checking for nulls
waseca.isnull().any()

In [ ]:
#looking at how many nulls there are: just 1
waseca.info()

In [ ]:
#looking for when a metric is null:
waseca[waseca['tmax'].isnull()]

In [ ]:
#this looks like a missed point of collection, noting the existence, but I'm not going to pull out 
waseca.plot()

In [ ]:
#looks like 30 is the only hole. I will note it's presence and possibly take it out later.

### Reading in observed evapotranspiration data to test model

In [ ]:
#reading in observed ET data
observed_et_data_waseca_corn_r204 = pd.read_excel('Data/ObservedET_Waseca_2007.xls', sheet_name = 'Corn_R204')
print("Corn Data frame shape:", observed_et_data_waseca_corn_r204.shape)
observed_et_data_waseca_corn_r204.head(2)

Initial data investigation: can already see existence of nulls and ugly column names

In [ ]:
observed_et_data_waseca_corn_r204.info()

determining the target datatype for each series. 

In [ ]:
#it looks like all columns should be numeric
observed_et_data_waseca_corn_r204.tail()

In [ ]:
#confirming ET not null values are also numeric
observed_et_data_waseca_corn_r204[observed_et_data_waseca_corn_r204['ET'].notnull()]

In [ ]:
#renaming columns and pulling off the first row:
renaming_waseca_corn_columns = {
        'Unnamed: 0': 'year', 
        'Unnamed: 1': 'day', 
        'Soil Moisture Content': 'soil_moist_cont_20_cm', 
        'Unnamed: 3': 'soil_moist_cont_40_cm',
        'Unnamed: 4': 'soil_moist_cont_80_cm', 
        'Unnamed: 5': 'soil_moist_cont_100_cm', 
        'ET': 'et_mm',
    }
observed_et_data_waseca_corn_r204.rename(columns=renaming_waseca_corn_columns,inplace=True)
print(observed_et_data_waseca_corn_r204.shape)
observed_et_data_waseca_corn_r204.drop(index=0,inplace=True)
print("Shape of df after dropping the first row:",observed_et_data_waseca_corn_r204.shape)
observed_et_data_waseca_corn_r204.head()

In [ ]:
#looking to see if there is any variance in year
observed_et_data_waseca_corn_r204['year'].value_counts()

In [ ]:
#checking to see if the day ever comes up more than once:
observed_et_data_waseca_corn_r204['day'].value_counts()

### Reading in weather data observed in 2007

In [ ]:
#reading in Weather data observed 2007
observed_weather_waseca_2007 = pd.read_excel('Data/WasecaWeather_2006and2007final.xls', sheet_name = '2007')
print(observed_weather_waseca_2007.shape)
observed_weather_waseca_2007.head()

In [ ]:
observed_weather_waseca_2007.info()

In [ ]:
observed_weather_waseca_2007.head(3)

In [ ]:
#renaming dict for column in observed_weather_waseca_2007 to be single line evaluations
observed_weather_waseca_2007_columns_rename_dict = {
    'Unnamed: 0': 'year', 
    'Unnamed: 1': 'day', 
    'Unnamed: 2': 'temp_max_celcius', 
    'Unnamed: 3': 'temp_min_celcius', 
    'Rain ': 'rain_runoff_site_mm',
    'Unnamed: 5': 'tile_site_mm', 
    'Pan Evaporation': 'pan_evaporation_observed_mm', 
    'Unnamed: 7': 'est_daily_mm', 
    'Unnamed: 8': 'radiation_megaj_msq_d',
    'Unnamed: 9': 'rhmax_percent', 
    'Unnamed: 10': 'rhmin_percent', 
    'Wind ': 'wind_speed_m_s'
}
observed_weather_waseca_2007.rename(columns=observed_weather_waseca_2007_columns_rename_dict,inplace=True)
print(observed_weather_waseca_2007.shape)
observed_weather_waseca_2007.drop(index=[0,1],inplace=True)
print("Shape of df after dropping the first and second row:",observed_weather_waseca_2007.shape)
observed_weather_waseca_2007.head()

In [ ]:
#to get this .iat to be the same number I had to minus the first number by 2 and the column stayed the same, but the .at works fine without it


In [ ]:
#updated
ΔTmin = observed_weather_waseca_2007.at[3,'temp_min_celcius']-observed_weather_waseca_2007.at[2,'temp_min_celcius']
ΔTmax = observed_weather_waseca_2007.at[3,'temp_max_celcius']-observed_weather_waseca_2007.at[2,'temp_max_celcius']
print(ΔTmin, ΔTmax)

### Calculating the transition probabilities based on the wet to dry trends

In [ ]:
rain = waseca[waseca['rain'].notnull()]['rain']#making sure we're ignoring the 30th day where data wasn't taken:
total_days = len(rain)
#calculating the amount of we days by counting the amount of days that have precip more than 0:
print("total_days: ", total_days)
#creating a counter for the number of instances the amount of rain isn't zero:
wet_days = 0
for rain_amount in rain:
    if rain_amount != 0:
        wet_days +=1
print("wet_days: ", wet_days)

#calculating the amount of dry days by taking the total amount of days and subtracting the wet days
dry_days = total_days-wet_days
print("dry_days: ", dry_days)

#calculating the probabillity of a wet day:
prob_wet_day = wet_days/total_days #See page 2-40 for explanation
print("prob_wet_day: ", prob_wet_day)
#calculating the probability of a dry day:
prob_dry_day = 1-prob_wet_day
print("prob_dry_day: ", prob_dry_day)

#initializing values for transition probability calculations
wet_to_wet = 0
wet_to_dry = 0
dry_to_wet = 1 #we can't compare to day negative one, and so we are assuming the day is dry to match the sums as seen in table 2.2
dry_to_dry = 0
for i in rain.index:
    #accounting for the end of the road (cannot do this calculation bc there is no next day in the dataset):
    if i == total_days:
        #do nothing
        pass
    else:
        j = i+1
        #accouting for 30 being out of commission:
        if j in waseca[waseca['rain'].isnull()].index:
            j+=1
        if(rain[i] !=0 and rain[j]!=0):
            wet_to_wet += 1
        elif(rain[i] !=0 and rain[j]==0):
            wet_to_dry +=1
        elif(rain[i] ==0 and rain[j]!=0):
            dry_to_wet +=1
        else:
            dry_to_dry +=1

print("wet_to_wet: ", wet_to_wet)
print("wet_to_dry: ", wet_to_dry)
print("dry_to_wet: ", dry_to_wet)
print("dry_to_dry: ", dry_to_dry)

#calculating the probability of a wet day to have a next wet day:
p_w_w = wet_to_wet/wet_days
print("p_w_w:", p_w_w)
#calculating the probability of a wet day to have a next dry day:
p_w_d = wet_to_dry/dry_days
print("p_w_d:", p_w_d)
#calculating the probability of a dry day to have a next wet day:
p_d_w = 1-p_w_w
print("p_d_w:",p_d_w)

#to make into a test later:
#Data Check: should be the same 
p_d_w2 = dry_to_wet/wet_days
print(p_d_w2)

#calculating the probability of a dry day to have a next dry day:
p_d_d = 1-p_w_d
print("p_d_d:", p_d_d)
#Data Check
#p_d_d2 = dry_to_dry/dry_days
#print(p_d_d2)

In [ ]:
#old values ==> related to hard coded coefficients?:
#lowercased the variables
total_days:  62
wet_days:  21
dry_days:  41
prob_wet_day:  0.3387096774193548
prob_dry_day:  0.6612903225806452
wet_to_wet:  8
wet_to_dry:  13
dry_to_wet:  13
dry_to_dry:  28
p_w_w: 0.38095238095238093
p_w_d: 0.3170731707317073
p_d_w: 0.6190476190476191
p_d_d: 0.6829268292682926

## Richards prediction calculations

In [ ]:
#make a function to calculate Richardson's prediction

#====================================================================================================
#Function to calculate the vday value to compare against the random value to determine if the day will be wet or dry
#==============================================================================================================
def vdaycalc(day,currentState):
        
    #Note: Assume coefficients are the same for rochester MN:Table. 2.4. Cosine Coefficient for Maximum Temperature Statistics, Rochester, MN.
    #Because the first day is dry, we will set the "Global variables" called only on the first iteration of the loop
    #we will use the coefficients correlated for the P(W/D) for initially:https://drive.google.com/file/d/1Njs-T9RO7uShIAJr-r1JKp6N8rbeRXLF/view?usp=sharing
    C0 = 0.225
    C1 = -0.059
    C2 = 0.008
    C3 = 0.008
                
    theta1= 0.236
    theta2= 0.981
    theta3= 0.489
    #these get overwritten when in the loop
    #Calculating period:
    T=365/(2*math.pi)
    if currentState == "wet": # P(W/W) 
        C0 = .392
        C1 = -.035
        C2 = -.055
        C3 = .01
            
        theta1= 0.392 
        theta2= -0.982
        theta3= -0.343 
            
    else: #Current state == dry P(W/D)
        C0 = 0.225
        C1 = -0.059
        C2 = 0.008
        C3 = 0.008
            
        theta1= 0.236
        theta2= 0.981
        theta3= 0.489
    #calculate the Richardson estimation of the parameters based on his eqn 2.5.11:
    vday=float(C0+C1*math.cos(day/T+theta1)+C2*math.cos(2*day/T+theta2)+C3*math.cos(3*day/T+theta3))
    return(vday)

# Calculate If the day is wet or dry
currentStatei = []
currentState = "dry" #chosen such that the above transition probabilities would sum correctly
precip = []
for day in range(366):
    #call the Richardson estimation function inside the loop
    vday=vdaycalc(day,currentState)
    randomdayvalue=random.random()
    if vday==randomdayvalue:
        print('something is wrong')
    elif randomdayvalue>vday:
        nextday = "dry"
        P=0
    else:
        #estimated based on the vday equation and the respective coefficient from the data given 
        mu = float(C0+C1*math.cos(day/T+theta1)+C2*math.cos(2*day/T+theta2)+C3*math.cos(3*day/T+theta3))
        #Changed the precip to fit a skewed normal estimation based on Bruce's office hours
        P=mu*float(skewnorm.rvs(a=2, size=1)) #mm
        #possible source of error: forcing to positive
        if P<0:
            P=abs(mu*float(skewnorm.rvs(a=2, size=1))) #mm
        nextday = "wet"
    precip.append(P)
    currentStatei.append(currentState)
    currentState=nextday
    
yearRecord = pd.DataFrame(
    {'Current State': currentStatei,
     'Precip': precip
    })

N = 365
t = numpy.linspace(0, 1, 365)

print(yearRecord)
#Simulated precip
fig = go.Figure(data=go.Scatter(x=t, y=precip, mode='markers'))

fig.show()

# Up next to correct:

## TDL:
* update variable names obv
* look at any leftover "unnamed" columns to be the cleaned df call version
* check and .iat and make sure they're universal and not dirty dataframe dependent
* look at other notebook and see if there are anymore work you have in there not in here

### Infiltration component

In [1]:
#predicting initial moisture content for boundary condition:
#May 39th is the 150th day of the year, pull 151 column for soil moisture information as initial
initialsoilmoisturecontent = float(observed_et_data_waseca_corn_r204.at[31,'soil_moist_cont_20_cm'])
#alternative initial estimation method:
#θi = θwp+0.5*(θfc-θwp) #initial state of the soil moisture

# Parameter estimation
#soil tank depth mm to correlate to the soil information for the Nicollet Clay Loam at .2 m
#soil information from table given in the site
D=200
θs=0.48
θfc=0.41
θwp=0.26
b=13.4
Ψb=108 #mm of H20
Ks = 55.2 #mm/d saturated conductivity
θi=initialsoilmoisturecontent #global variable to use for the first iteration in the look
#======================================================================================================

def calcfp(day):
    """ 
    Calculating net infiltration and percolation
    """
    Net_Inf_Per = (θfc-θi)*D
    if precip[day]<Net_Inf_Per:
        #precip[day] = precipitation simulated for that day
        Net_Inf_Per = precip[day]
    return Net_Inf_Per

#=====================================================================================================
def calcRn(day):
    """ 
    Calculating Net Radiation
    assuming the same parameters given on page 6-50 for grass conditions
    """
    bo = -0.52 #MJ/m2/d 
    b1 = 0.75 #MJ/m2/d
    row = day-118
    Rsi=float(observed_weather_waseca_2007.at[row,'Unnamed: 8'])
    Rn = bo+b1*Rsi
    return Rn

#====================================================================================================
def calcΨRET(day):
    """ 
    Compute log profile resistance constant:
    Calculating RET Using Penmans, referencing page  6-50; 6-82 for algorithm summary.
    referencing Kpa eqn 6.4.2 in book
    """
    γc = 0.066 #kPa/K = 0.66 mbar/K.
    L = 2453 #MJ m-3
    G=0 #for a daily time step
    cp = 1 #kJ kg-1 K-1
    rowcurrentday=day-118
    rowyesterday=day-119
    ΔTmin = observed_weather_waseca_2007.at[rowcurrentday,'temp_min_celcius']-observed_weather_waseca_2007.at[rowyesterday,'temp_min_celcius']
    ΔTmax = observed_weather_waseca_2007.at[rowcurrentday,'temp_max_celcius']-observed_weather_waseca_2007.at[rowyesterday,'temp_max_celcius']
    Δhat = (ΔTmin+ΔTmax)/2 #C
    U2= observed_weather_waseca_2007.iat[rowcurrentday,11] #km d-1
    fu= 6.43*(1+0.0062*U2) #MJ m-2 d-1 kPa-1
    esatmin=((observed_weather_waseca_2007.at[rowcurrentday,'temp_min_celcius']+273.15)/273.16)**(-4.811)*math.exp(24.134-6726.73/(observed_weather_waseca_2007.at[rowcurrentday,'temp_min_celcius']+273.15)) #Kpa eqn 6.4.2
    esatmax=((observed_weather_waseca_2007.at[rowcurrentday,'temp_max_celcius']+273.15)/273.16)**(-4.811)*math.exp(24.134-6726.73/(observed_weather_waseca_2007.at[rowcurrentday,'temp_max_celcius']+273.15)) #Kpa eqn 6.4.2
    rhmax=observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 9']/100
    rhmin=observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 10']/100
    dailyvaporpressuredeficit=(esatmax+esatmax)/2-(esatmin*rhmax+esatmax*rhmin)/2
    ΨRET=Δhat/(Δhat+γc)*(calcRn(day)-G)+γc/(Δhat+γc)*fu*dailyvaporpressuredeficit
    return ΨRET
#=====================================================================================================
def calcRETpenm(day):
    RET=calcΨRET(day)/L
    return RET
#=====================================================================================================
def calcET(day):
    """
    calculating soil moisture availability factor
    Coefficient of rapid evaporation loss
    look up Kr: http://www.fao.org/3/X0490e/x0490e0c.htm from generalized figure
    """
    if θi>θfc:
        Kw=1
        #Kr=1
    elif θi<θwp:
        Kw=0
        #Kr=.4
    else:
        Aw=100*(θi-θwp)/(θfc-θwp)
        Kw=np.log(Aw+1)/np.log(101)
        #Kr=.7
        
    #crop coefficient for corn
    Kc = .3 #table 6.6
    
    
    ET=(Kc*Kw)*calcRETpenm(day)
    return ET
    

#=====================================================================================================
def calcθi(day,θi):
    """ 
    alculating soil moisture content Penman
    """
    θi=θi+calcfp(day)/D-calcET(day)/D
    return θi

#====================================================================================================
def calcETpanmeth(day):
    """
    Pan Evap Method
    """
    Kc = .3 #table 6.6
    row=day-118
    RET=cp*observed_weather_waseca_2007.at[row,'Unnamed: 7']
        
    PET=Kc*RET
    if θi>θfc:
        Kw=1
        #Kr=1
    elif θi<θwp:
        Kw=0
       #Kr=.4
    else:
        Aw=100*(θi-θwp)/(θfc-θwp)
        Kw=np.log(Aw+1)/np.log(101)
        Kr=.7
        
    #crop coefficient for corn
    ET=Kw*PET
    return ET

#=====================================================================================================
def calcθipan(day,θi):
    """ 
    Calculating soil moisture content Pan evap method
    """
    θi=θi+calcfp(day)/D-calcETpanmeth(day)/D
    return θi

NameError: name 'observed_et_data_waseca_corn_r204' is not defined

In [ ]:

ETpan= []
soilmoisturepan = []
soilmoistureContent = []
ETlist = []
infilperclist=[]

for day in range(152,273,1):
    ET = 1000*calcET(day)
    ETlist.append(ET)
    fp=calcfp(day)
    infilperclist.append(fp)
    θi = calcθi(day,θi)
    soilmoistureContent.append(θi)
    ETPan = calcETpanmeth(day)
    ETpan.append(ETPan)
    θPan =  calcθipan(day,θi)
    soilmoisturepan.append(θPan)
    
df = pd.DataFrame({'ET Penman':ETlist, 'θ Penman':soilmoistureContent, 'ET Pan':ETpan, 'θ Pan':soilmoisturepan})
print(df)

observedsoilmoisture= observed_et_data_waseca_corn_r204['Unnamed: 3'].values.tolist()
observedET= observed_et_data_waseca_corn_r204['ET'].values.tolist()


t = numpy.linspace(152, 273, 121)
soilmoisture = go.Figure(data=go.Scatter(x=t, y=soilmoistureContent, name='Soil Moisture Content Penman', mode='markers'))
soilmoisture.add_trace(go.Scatter(x=t, y=soilmoisturepan, name='Soil Moisture Content Pan Evap', mode='markers'))
soilmoisture.add_trace(go.Scatter(x=t, y=observedsoilmoisture, name='Soil Moisture Content observed', mode='markers'))
soilmoisture.show()

ET = go.Figure(data=go.Scatter(x=t, y=ETlist, name='ET Penman', mode='markers', marker_color='rgba(152, 0, 0, .8)'))
ET.add_trace(go.Scatter(x=t, y=ETpan, name='ET pan', mode='markers'))
ET.add_trace(go.Scatter(x=t, y=observedET, name='ET Observed', mode='markers'))
ET.show()

#ET.write_html(r'C:\Users\clair\Downloads\ETgraph.html')
#soilmoisture.write_html(r'C:\Users\clair\Downloads\soilmoisturegraph.html')

In [ ]:
#With actual precip data

initialsoilmoisturecontent = float(observed_et_data_waseca_corn_r204.at[31,'Soil Moisture Content'])
#alternative initial estimation method:
#θi = θwp+0.5*(θfc-θwp) #initial state of the soil moisture
#=====================================================================================================
# Parameter estimation
#soil tank depth mm to correlate to the soil information for the Nicollet Clay Loam at .2 m
#soil information from table given in the site
D=200
θs=0.48
θfc=0.41
θwp=0.26
b=13.4
Ψb=108 #mm of H20
Ks = 55.2 #mm/d saturated conductivity
θi=initialsoilmoisturecontent #global variable to use for the first iteration in the look
#======================================================================================================

#====================================================================================================
#Calculating net infiltration and percolation
#=====================================================================================================
def calcfp(day):
    Net_Inf_Per = (θfc-θi)*D
    if actualprecip[day-150]<Net_Inf_Per:
        #precip[day] = precipitation simulated for that day
        Net_Inf_Per = precip[day]
    return Net_Inf_Per

#====================================================================================================
#Calculating Net Radiation
#assuming the same parameters given on page 6-50 for grass conditions
#=====================================================================================================
bo = -0.52 #MJ/m2/d 
b1 = 0.75 #MJ/m2/d
def calcRn(day):
    row = day-118
    Rsi=float(observed_weather_waseca_2007.at[row,'Unnamed: 8'])
    Rn = bo+b1*Rsi
    return Rn

#====================================================================================================
#Calculating RET Using Penmans
#referencing page  6-50; 6-82 for algorithm summary
#=====================================================================================================
γc = 0.066 #kPa/K = 0.66 mbar/K.
L = 2453 #MJ m-3
G=0 #for a daily time step
cp = 1 #kJ kg-1 K-1
#Compute log profile resistance constant:
def calcΨRET(day):
    rowcurrentday=day-118
    rowyesterday=day-119
    ΔTmin = observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 3']-observed_weather_waseca_2007.at[rowyesterday,'Unnamed: 3']
    ΔTmax = observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 2']-observed_weather_waseca_2007.at[rowyesterday,'Unnamed: 2']
    Δhat = (ΔTmin+ΔTmax)/2 #C
    U2= observed_weather_waseca_2007.iat[rowcurrentday,11] #km d-1
    fu= 6.43*(1+0.0062*U2) #MJ m-2 d-1 kPa-1
    esatmin=((observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 3']+273.15)/273.16)**(-4.811)*math.exp(24.134-6726.73/(observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 3']+273.15)) #Kpa eqn 6.4.2
    esatmax=((observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 2']+273.15)/273.16)**(-4.811)*math.exp(24.134-6726.73/(observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 2']+273.15)) #Kpa eqn 6.4.2
    rhmax=observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 9']/100
    rhmin=observed_weather_waseca_2007.at[rowcurrentday,'Unnamed: 10']/100
    dailyvaporpressuredeficit=(esatmax+esatmax)/2-(esatmin*rhmax+esatmax*rhmin)/2
    ΨRET=Δhat/(Δhat+γc)*(calcRn(day)-G)+γc/(Δhat+γc)*fu*dailyvaporpressuredeficit
    return ΨRET

def calcRETpenm(day):
    RET=calcΨRET(day)/L
    return RET

def calcET(day):
    
    #calculating soil moisture availability factor
    #Coefficient of rapid evaporation loss
    #look up Kr: http://www.fao.org/3/X0490e/x0490e0c.htm from generalized figure
    if θi>θfc:
        Kw=1
        #Kr=1
    elif θi<θwp:
        Kw=0
        #Kr=.4
    else:
        Aw=100*(θi-θwp)/(θfc-θwp)
        Kw=np.log(Aw+1)/np.log(101)
        #Kr=.7
        
    #crop coefficient for corn
    Kc = .3 #table 6.6
    
    
    ET=(Kc*Kw)*calcRETpenm(day)
    return ET
    

#====================================================================================================
#Calculating soil moisture content Penman
#=====================================================================================================
def calcθi(day,θi):
    θi=θi+calcfp(day)/D-calcET(day)/D
    return θi

#Pan Method
Kc = .3 #table 6.6
#====================================================================================================
#Pan Evap Method
#=====================================================================================================
def calcETpanmeth(day):
    row=day-118
    RET=cp*observed_weather_waseca_2007.at[row,'Unnamed: 7']
        
    PET=Kc*RET
    if θi>θfc:
        Kw=1
        #Kr=1
    elif θi<θwp:
        Kw=0
       #Kr=.4
    else:
        Aw=100*(θi-θwp)/(θfc-θwp)
        Kw=np.log(Aw+1)/np.log(101)
        Kr=.7
        
    #crop coefficient for corn
    ET=Kw*PET
    return ET

#====================================================================================================
#Calculating soil moisture content Pan evap method
#=====================================================================================================
def calcθipan(day,θi):
    θi=θi+calcfp(day)/D-calcETpanmeth(day)/D
    return θi


ETpan= []
soilmoisturepan = []
soilmoistureContent = []
ETlist = []
infilperclist=[]

for day in range(152,273,1):
    ET = 1000*calcET(day)
    ETlist.append(ET)
    fp=calcfp(day)
    infilperclist.append(fp)
    θi = calcθi(day,θi)
    soilmoistureContent.append(θi)
    ETPan = calcETpanmeth(day)
    ETpan.append(ETPan)
    θPan =  calcθipan(day,θi)
    soilmoisturepan.append(θPan)
    
df = pd.DataFrame({'ET Penman':ETlist, 'θ Penman':soilmoistureContent, 'ET Pan':ETpan, 'θ Pan':soilmoisturepan})
print(df)

observedsoilmoisture= observed_et_data_waseca_corn_r204['Unnamed: 3'].values.tolist()
observedET= observed_et_data_waseca_corn_r204['ET'].values.tolist()


t = numpy.linspace(152, 273, 121)
soilmoisture = go.Figure(data=go.Scatter(x=t, y=soilmoistureContent, name='Soil Moisture Content Penman', mode='markers'))
soilmoisture.add_trace(go.Scatter(x=t, y=soilmoisturepan, name='Soil Moisture Content Pan Evap', mode='markers'))
soilmoisture.add_trace(go.Scatter(x=t, y=observedsoilmoisture, name='Soil Moisture Content observed', mode='markers'))
soilmoisture.show()

ET = go.Figure(data=go.Scatter(x=t, y=ETlist, name='ET Penman', mode='markers', marker_color='rgba(152, 0, 0, .8)'))
ET.add_trace(go.Scatter(x=t, y=ETpan, name='ET pan', mode='markers'))
ET.add_trace(go.Scatter(x=t, y=observedET, name='ET Observed', mode='markers'))
ET.show()

#ET.write_html(r'C:\Users\clair\Downloads\ETgraph.html')
#soilmoisture.write_html(r'C:\Users\clair\Downloads\soilmoisturegraph.html')